## Imports

In [25]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from ipaddress import *
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import SimpleRNN, Dense, Embedding, Dropout, Flatten, SpatialDropout1D, LSTM,Input, Concatenate, Reshape, InputLayer, concatenate
from tensorflow.keras.regularizers import l1, l2, l1_l2
import keras
import re
import tldextract
import warnings
warnings.filterwarnings("ignore")
sns.set_style(style='white') 

## Data Transforming

In [26]:

data=pd.read_csv('cleaned_data.csv')
df=data[['url','status']]
data=data.drop('url',axis=1)

In [27]:
data.head()

,status,url_length,num_special_chars,num_digits,num_dots,num_slash,num_and,num_percent,domain_length,num_digits_in_domain,has_subdomain,is_HTTPS,depth,top_domain_encoded,domain_encoded
0,0,39,0,7,2,0,0,0,12,0,1,0,1,225,111707
1,0,36,0,6,2,0,0,0,12,0,1,0,1,225,111707
2,0,12,0,5,1,0,0,0,5,5,0,0,1,878,10
3,0,35,0,4,2,0,0,0,12,0,1,0,1,225,111707
4,0,20,0,5,2,0,0,0,8,0,1,0,1,225,291146


In [28]:
y=data['status']
vectorizer = TfidfVectorizer(analyzer='char')
X=vectorizer.fit_transform(df.url)

In [29]:
X.shape

(798463, 266)

In [30]:
X=X.toarray()
print(X.shape)
X=X.reshape(X.shape[0],14,19)

(798463, 266)


In [31]:
X_train, X_test, y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=54)



In [32]:
X_train.shape

(558924, 14, 19)

In [33]:
# X_train = pad_sequences(X_train, padding='post', truncating='post')
# X_test = pad_sequences(X_test, padding='post', truncating='post')


## LSTM

In [34]:
tf.keras.backend.clear_session()
tf.random.set_seed(50)
Lstm  = Sequential()
Lstm.add(LSTM(128,input_shape=(14,19),return_sequences=False))
Lstm.add(Dropout(0.2))
Lstm.add(Dense(24,activation='relu'))
Lstm.add(Dense(1,activation ='sigmoid'))
# Lstm.add(Dense(1,activation ='sigmoid',kernel_regularizer=l1(0.01), bias_regularizer=l2(0.01)))

In [35]:
Lstm.compile(loss ='binary_crossentropy',optimizer='adam', metrics=['accuracy', 'precision', 'recall'])

In [36]:
num_epochs =30
early_stop = EarlyStopping(monitor='val_loss',patience=3, verbose=1, restore_best_weights=True)
historyL=Lstm.fit(X_train, y_train,epochs = num_epochs,validation_data=(X_test,y_test),callbacks =[early_stop],verbose =2)
# around8 minutes an epeoch when split 
# reached around88

Epoch 1/30
17467/17467 - 225s - 13ms/step - accuracy: 0.8715 - loss: 0.3013 - precision: 0.8549 - recall: 0.9149 - val_accuracy: 0.8963 - val_loss: 0.2509 - val_precision: 0.8629 - val_recall: 0.9582
Epoch 2/30
17467/17467 - 270s - 15ms/step - accuracy: 0.9051 - loss: 0.2323 - precision: 0.8908 - recall: 0.9375 - val_accuracy: 0.9142 - val_loss: 0.2106 - val_precision: 0.8935 - val_recall: 0.9532
Epoch 3/30
17467/17467 - 255s - 15ms/step - accuracy: 0.9150 - loss: 0.2100 - precision: 0.9037 - recall: 0.9414 - val_accuracy: 0.9205 - val_loss: 0.1972 - val_precision: 0.9055 - val_recall: 0.9504
Epoch 4/30
17467/17467 - 240s - 14ms/step - accuracy: 0.9207 - loss: 0.1972 - precision: 0.9102 - recall: 0.9449 - val_accuracy: 0.9236 - val_loss: 0.1895 - val_precision: 0.9117 - val_recall: 0.9491
Epoch 5/30
17467/17467 - 248s - 14ms/step - accuracy: 0.9249 - loss: 0.1873 - precision: 0.9154 - recall: 0.9471 - val_accuracy: 0.9267 - val_loss: 0.1817 - val_precision: 0.9138 - val_recall: 0.9529
